# Elementos Necesarios

## Librerias

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2
from importlib import reload
import pandas as pd
import sympy as sp
from scipy.optimize import curve_fit
import scipy.stats as stats
from scipy.stats import norm
from scipy.integrate import quad
plt=reload(plt)

## Funciones

### Leer Archivos txt

In [ ]:
def leer_txt(nombre_archivo):
    f = open(f"{nombre_archivo}.txt",'r')
    lines = f.readlines()[4:-1]

    tiempo = [] #segundos
    newtons = []

    for line in lines:
        tiempo.append(float(line.split("\t")[0].replace(',', '.')))
        newtons.append(float(line.split("\t")[1].split("\n")[0].replace(',', '.')))

    tiempo = np.array(tiempo)
    newtons = np.array(newtons)

    return tiempo, newtons

### Leer xlms

In [ ]:
"""
datos = pd.read_excel('NOMBRE DEL ARCHIVO.xlsx')
for i in range(1, 10):
    columna_longitud = f'longitud{i}'
    pasadas = []
    for j in range(0, 5):
        if datos[columna_longitud][j] != 'NaN':
            pasadas.append(datos[columna_longitud][j])
    longitudes_generales.append(np.mean(pasadas))
"""

"\ndatos = pd.read_excel('NOMBRE DEL ARCHIVO.xlsx')\nfor i in range(1, 10):\n    columna_longitud = f'longitud{i}'\n    pasadas = []\n    for j in range(0, 5):\n        if datos[columna_longitud][j] != 'NaN':\n            pasadas.append(datos[columna_longitud][j])\n    longitudes_generales.append(np.mean(pasadas))\n"

### Derivadas Parciales

In [ ]:
def derivada_parcial(variable, funcion, valores=None):
    derivada = sp.diff(funcion, variable)

    if valores != None and type(valores) == dict:
        return derivada.subs(valores)
    
    elif valores != None and type(valores) == list:
        derivadas_evaluadas = []

        for i in valores:
            evaluar = sp.lambdify(variable, derivada)
            derivadas_evaluadas.append(evaluar(i))
        derivadas_evaluadas = np.array(derivadas_evaluadas)
        
        return derivadas_evaluadas

    elif valores != None and (type(valores) == int or type(valores) == float): return derivada.evalf(subs={variable: valores})

    else: return derivada

### Cuadrados Mínimos

In [ ]:
#definimos el parametro a
def a(N, xi, yi ):
    numerador_a = (N * np.sum(xi * yi)) - ((np.sum(xi)) * (np.sum(yi))) 
    denominador_a = ((N*np.sum(xi**2))) - ((np.sum(xi)**2))
    return numerador_a/denominador_a
#definimos el parametro b
def b(N,xi,yi):
    numerador_b = (np.sum(xi**2))*(np.sum(yi)) - (np.sum(xi)*np.sum(xi*yi))
    denominador_b = ((N*np.sum(xi**2))) - ((np.sum(xi)**2))
    return numerador_b/denominador_b 
#definimos f(xi)= axi +b
def f(N,xi,yi): return a(N,xi,yi)* xi + b(N,xi,yi)

def sigma_a(N,xi,yi):
    delta_a= ((N*np.sum(xi**2))) - ((np.sum(xi)**2))
    cociente= N/delta_a
    sigma_ya=np.std(yi)
    return sigma_ya*np.sqrt(cociente)
def sigma_b(N,xi,yi):
    delta_b= ((N*np.sum(xi**2))) - ((np.sum(xi)**2))
    cociente= (np.sum(xi**2))/delta_b
    sigma_yb=np.std(yi)
    return sigma_yb*np.sqrt(cociente)
   
    #N= cantidad de mediciones 
    #donde Xi Yi son los datos medidos 

### Cuadrados Mínimos Ponderados

In [3]:
#definimos el parametro a
def ap(xi, yi, sigma_yi):
    wi = 1 / (sigma_yi**2)
    numerador_a = (np.sum(wi) * np.sum(xi * yi * wi)) - ((np.sum(xi * wi)) * (np.sum(yi * wi))) 
    denominador_a = (np.sum(wi) * np.sum(wi * (xi**2)) - (np.sum(wi * xi)**2))
    return (numerador_a / denominador_a)

#definimos el parametro b
def bp(xi,yi, sigma_yi):
    wi = 1 / (sigma_yi**2)
    numerador_b = (np.sum(wi * (xi**2))) * (np.sum(wi * yi)) - (np.sum(xi * wi) * np.sum(xi * yi * wi))
    denominador_b = (np.sum(wi) * np.sum(wi * (xi**2)) - (np.sum(wi * xi)**2))
    return numerador_b / denominador_b 

#definimos f(xi)= axi + b
def fp(xi,yi,sigma_yi): return ap(xi,yi,sigma_yi)* xi + bp(xi,yi,sigma_yi)

#definimos sigmas de a y b
def sigma_ap(xi,yi,sigma_yi):
    wi = 1 / (sigma_yi**2)
    denominador_a = (np.sum(wi) * np.sum(wi * (xi**2)) - (np.sum(wi * xi)**2))
    radicando = (np.sum(wi)) / (denominador_a)
    return np.sqrt(radicando)

def sigma_bp(xi,yi,sigma_yi):
    wi = 1 / (sigma_yi**2)
    denominador_b = (np.sum(wi) * np.sum(wi * (xi**2)) - (np.sum(wi * xi)**2))
    radicando = (np.sum(wi * xi**2)) / denominador_b
    return np.sqrt(radicando)

### Coeficiente de Pearson $(r^2)$

In [ ]:
def coef_person(xi, yi):
    if len(xi) == len(yi):
        N = len(xi)
        numerador_cp = ((N * (np.sum(xi * yi))) - (np.sum(xi) * np.sum(yi))) **2
        denominador_cp = ((N * np.sum(xi**2)) - (np.sum(xi))**2) * ((N * np.sum(yi**2)) - (np.sum(yi))**2)
        return np.sqrt(numerador_cp / denominador_cp)
    else:
        return print(f'Los valores en x son distinto que los y: |cantidad en x:{len(xi)}| |cantidad en y:{len(yi)}|')

### Cifras Significativas

In [ ]:
def redondearA1CS(datos):
    # Requiere: tipo numero
    def redondearUnNumeroA1CS(numero):
        cifras = 0
        n = numero
        if numero < 10:
            while n < 1:
                cifras += 1
                n *= 10

            # La funcion round de python no redondea el 5 para arriba
            # Este es el workaround para eso
            proximoDecimal = int(numero * (10**(cifras+1)) % 10)
            if proximoDecimal == 5:
                numero += 1 * (10**-cifras)

            n = round(numero,cifras)

            return n, cifras

        if numero >= 10:
            while n >= 10:
                cifras +=1
                n /= 10
            n = round(numero, -cifras)
            return n, -cifras
    
    if hasattr(datos, '__iter__'):
        datosRedondeados = []
        listaDecimales = []
        for valor in datos:
            valor_aux, decimales_aux = redondearUnNumeroA1CS(valor)
            datosRedondeados.append(valor_aux)
            listaDecimales.append(decimales_aux)

        return datosRedondeados, listaDecimales
    
    # Si no es una lista, asume que es un numero
    return redondearUnNumeroA1CS(datos)

### P-value

In [ ]:
def p_value(yi, fxi, sigma_yi, mediciones, parametros):
    p_libres = mediciones - parametros
    
    def chi_2(yi, fxi, sigma_yi):
        numerador_chi_2 = yi - fxi
        return np.sum((numerador_chi_2 / sigma_yi)**2)
    
    return chi2.sf(chi_2(yi, fxi, sigma_yi), p_libres)

### Curvefit funciones con parámetros no lineales

In [ ]:
def obtener_curvefit(funcion, dato1, dato2, condiciones_iniciales):
    popt, pcov = curve_fit(funcion, dato1, dato2, p0=[condiciones_iniciales])
    errores = np.sqrt(np.diag(pcov))
    return popt, errores

### Gauss

In [ ]:
def gauss(x):
    media = np.mean(x)
    producto_gauss = 1 / ((np.sqrt(2*np.pi)) * np.std(x))
    exp_gauss = -(x - media)**2 / (2*(np.std(x))**2)
    return producto_gauss*np.exp(exp_gauss)

def integral_campana_gauss(media, incerteza, a, b):
    # Definir la función de distribución normal (campana de Gauss)
    gaussiana = norm(loc=media, scale=incerteza)
    
    # Definir la función a integrar
    integrando = lambda x: gaussiana.pdf(x)
    
    # Calcular la integral de la función entre los puntos a y b
    resultado, error = quad(integrando, a, b)
    
    return resultado

# Desarrollo del parcial